In [1]:
import pandas as pd
from rdkit.Chem import PandasTools, Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Descriptors
import seaborn as sns
from IPython.display import HTML
from tqdm import tqdm
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.metrics import silhouette_score
pKi_threshold = 4

Failed to patch pandas - PandasTools will have limited functionality


In [2]:
def butina_cluster(mol_list,cutoff=0.35):
    fp_list = [rdmd.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) for m in mol_list]
    dists = []
    nfps = len(fp_list)
    for i in tqdm(range(1, nfps)):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i],fp_list[:i])
        dists.extend([1-x for x in sims])
    mol_clusters = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    cluster_id_list = [0]*nfps
    print(len(cluster_id_list))
    for idx,cluster in enumerate(mol_clusters,1):
        for member in cluster:
            cluster_id_list[member] = idx
    return cluster_id_list, mol_clusters

In [ ]:
import os
_data_path = os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "exps", "data", "RDkit_data", "BL_Sets_data_rdkitblog.csv")
df = pd.read_csv(_data_path, sep=",")
df = df[df["pKi"]>pKi_threshold]
PandasTools.AddMoleculeColumnToFrame(df,smilesCol="smiles")
from time import time
st = time()
y = butina_cluster(df.ROMol.values)
print("Time: ", time()-st)

Failed to patch pandas - unable to change molecule rendering


In [5]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator
from classix import CLASSIX
# Setting up the Hyperparameters

regenerateFromScratch = True   # generate Morgan Fingerprints can take a while, so we save
pKi_threshold = 4
tanimoto_radius = 0.33
manhattan_radius = 1.3
mergeScale = 1.33
minPts = 50

# Paths relative to this notebook's location
_data_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "exps", "data", "RDkit_data")
_csv_path = os.path.join(_data_dir, "BL_Sets_data_rdkitblog.csv")
_npy_path = os.path.join(_data_dir, "BL_Sets.npy")

# Loading the Data
if regenerateFromScratch:
    # Data should be generated from scratch if the pKi threshold is changed.
    
    fp_gen = rdFingerprintGenerator.GetMorganGenerator(
        radius=2,  # atom radius to consider for hashing
        fpSize=1024  # bit-length of vector (smaller will lead to more collissions during hashing) 
    )
    df = pd.read_csv(_csv_path, sep=",")
    df["mol"] = df["smiles"].apply(Chem.MolFromSmiles)
    df = df[df["pKi"]>pKi_threshold]
    data = np.vstack(df["mol"].apply(fp_gen.GetFingerprintAsNumPy))
    np.save(_npy_path, data, allow_pickle=True)
else:
    df = pd.read_csv(_csv_path, sep=",")
    df = df[df["pKi"]>pKi_threshold]
    data = np.load(_npy_path)

cluster_membership = df["target_chembl_id"].astype("category").cat.codes
true_labels = np.array(cluster_membership)

# Initialise the models
clx_t = CLASSIX(metric="tanimoto", radius=tanimoto_radius, minPts=minPts, mergeScale=mergeScale)

data = data.astype(np.int32)
# Run Tanimoto Clustering
clx_t.fit(data)

CLASSIX(radius=0.33, minPts=50, group_merging='distance')


Aggregation: 100%|██████████| 90498/90498 [00:26<00:00, 3392.43it/s] 


Tanimoto merging completed: 214 clusters


CLASSIX(radius=0.33, minPts=50, group_merging='distance')

In [6]:
print(len(set(clx_t.labels_)))
print("ARI: ", ari(true_labels, clx_t.labels_))

214
ARI:  0.13593513169277388


In [ ]:
# for i in range(len(set(clx_m.labels))):
#     print(f"Cluster {i}: ", np.sum(clx_m.labels==i))
for i in range(len(set(clx_t.labels))):
    print(f"Cluster {i}: ", np.sum(clx_t.labels==i))

In [ ]:
for i in range(len(set(true_labels))):
    print(f"Cluster {i}: ", np.sum(true_labels==i))
# for i in range(len(set(clx_t.labels))):
#     print(f"Cluster {i}: ", np.sum(clx_t.labels==i))

In [106]:
from copy import deepcopy
print(len(set(clx_t.labels)))
print("ARI: ", ari(true_labels, clx_t.labels))
s = 0
for i in range(10):
    clx_labels = deepcopy(clx_t.labels)
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    # data = data[indices[::10]]
    # true_labels = true_labels[indices[::10]]
    st+=silhouette_score(data[indices[::10]], true_labels[indices[::10]], metric='jaccard')
    s+=silhouette_score(data[indices[::10]], clx_labels[indices[::10]], metric='jaccard')
print(s/10)
print(st/10)

184
ARI:  0.13043280581199362


/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric ja

0.059627973088725285
173263250.08458632


In [ ]:
from sklearn.cluster import DBSCAN
import time

start = time.time()
db = DBSCAN(eps=0.33, min_samples=5, metric='jaccard' ).fit(data)
end = time.time()
print("DBSCAN took", end-start, "seconds")
ari_score = ari(true_labels, db.labels_)

/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric ja

DBSCAN took 5064.842609882355 seconds


NameError: name 'db_scan_time' is not defined

In [3]:
# Run Manhattan Clustering

m_data = np.array(data, dtype=np.float32) # Need to convert data to float for Manhattan
clx_m.fit(m_data)


OWN AGGREGATION


100%|██████████| 90498/90498 [00:49<00:00, 1821.30it/s]  


  aggregation time: 49.70823526382446
  search time: 0.13016462326049805
  ips time: 49.182782888412476
  merging time: 0.57818603515625
 minPts Merging
small clusters [1 2]
CS:  [9.0493e+04 4.0000e+00 1.0000e+00]
label_sp_copy_2 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0

NameError: name 'CS' is not defined

In [6]:
clx_t.explain()

The data was clustered into 16401 groups. These were further merged into 184 clusters.


In [3]:
clx_m.minPts

50

In [72]:
from sklearn.metrics import silhouette_score

# score_data = silhouette_score(data[::100], clx_t.labels[::100], metric='jaccard')
score_data = silhouette_score(data[::100], db.labels_[::100], metric='jaccard')

/home/research/nlagroup/.local/lib/python3.11/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [73]:
score_data

0.05272480594421823

In [10]:
vec = np.random.rand(1024,1)
print(vec.shape)
probe = data@vec

print(len(np.unique(probe)))

(1024, 1)
51103
